In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from keras.optimizers import Adam
import librosa.feature
import os
import librosa
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import time
from tqdm import tqdm

In [4]:
data_folder = 'data_set'
subfolders = ['awake', 'belly_pain', 'burping', 'discomfort', 'hug', 'hungry', 'tired']

In [5]:
def extract_features(audio, sr, max_length=700):
    with tqdm(total=4, desc="Extracting Features", unit="feature") as pbar:
        # Извлечение MFCC признаков
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)
        pbar.update(1)

        # Извлечение Pitch признаков
        pitches, magnitudes = librosa.piptrack(y=audio, sr=sr)
        pitch = []
        for t in range(pitches.shape[1]):
            index = magnitudes[:, t].argmax()
            pitch.append(pitches[index, t])
        pitch = np.array(pitch)
        pitch = pitch[pitch > 0]
        pitch = pitch.reshape(1, -1)  # Изменение формы на (1, n)
        pbar.update(1)

        # Delta MFCC
        delta_mfcc = librosa.feature.delta(mfccs)
        pbar.update(1)

        # Delta Delta MFCC
        delta2_mfcc = librosa.feature.delta(mfccs, order=2)
        pbar.update(1)

        # Находим минимальную длину среди всех признаков
        min_length = min(mfccs.shape[1], pitch.shape[1], delta_mfcc.shape[1], delta2_mfcc.shape[1])

        # Обрезаем каждый признак до минимальной длины
        mfccs = mfccs[:, :min_length]
        pitch = pitch[:, :min_length]
        delta_mfcc = delta_mfcc[:, :min_length]
        delta2_mfcc = delta2_mfcc[:, :min_length]

        # Объединение признаков в один массив
        combined_features = np.vstack((mfccs, pitch, delta_mfcc, delta2_mfcc))

        # Обрезаем или дополняем нулями, чтобы получить фиксированный размер признаков
        if combined_features.shape[1] < max_length:
            pad_width = max_length - combined_features.shape[1]
            combined_features = np.pad(combined_features, ((0, 0), (0, pad_width)), 'constant')
        elif combined_features.shape[1] > max_length:
            combined_features = combined_features[:, :max_length]

        combined_features = combined_features.T  # Транспонирование для соответствия форме входных данных
    
#     print(combined_features)
    return combined_features

In [6]:
def load_and_preprocess_data():
    X = []  # Данные
    y = []  # Метки
    sr_list = []  # Частоты дискретизации

    total_files = sum([len(files) for r, d, files in os.walk(data_folder)])

    with tqdm(total=total_files, desc="Loading Data", unit="file") as pbar:
        for subfolder in subfolders:
            folder_path = os.path.join(data_folder, subfolder)
            for filename in os.listdir(folder_path):
                file_path = os.path.join(folder_path, filename)
                audio, sr = librosa.load(file_path)
                X.append(audio)
                sr_list.append(sr)
                y.append(subfolder)
                pbar.update(1)

    print("Encoding labels...")  # Чтобы пользователь знал, что происходит кодирование меток
    # Кодирование меток в числовые значения
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)

    # One-hot кодирование
    one_hot_encoder = OneHotEncoder(sparse=False)
    y_one_hot = one_hot_encoder.fit_transform(y_encoded.reshape(-1, 1))
    
#     print(X, sr_list, y_one_hot)
    return X, sr_list, y_one_hot

In [7]:
X, sr_list, y = load_and_preprocess_data()

Loading Data: 100%|██████████| 1426/1426 [00:11<00:00, 125.12file/s]
C:\Users\Buga\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Encoding labels...
[array([ 0.00079143, -0.00015322, -0.00080607, ...,  0.02129505,
        0.01183489,  0.00271348], dtype=float32), array([0.02598968, 0.03339507, 0.03181162, ..., 0.00522077, 0.00389671,
       0.        ], dtype=float32), array([-2.7120446e-05, -2.3509672e-05, -6.6363747e-05, ...,
        7.3920237e-03,  8.1988741e-03,  5.2069514e-03], dtype=float32), array([-0.00671496, -0.00734055, -0.00531321, ..., -0.10902438,
       -0.05949304,  0.        ], dtype=float32), array([-0.00552218, -0.00643346, -0.00688163, ..., -0.00366105,
       -0.00305783, -0.00231504], dtype=float32), array([0.00038065, 0.00094552, 0.00142655, ..., 0.0089041 , 0.00908294,
       0.        ], dtype=float32), array([ 2.68621545e-04,  9.32184630e-05, -4.75061970e-04, ...,
        7.69248903e-02,  1.24024495e-01,  8.59322697e-02], dtype=float32), array([-7.4415948e-05, -8.5632910e-04,  4.4318903e-04, ...,
        6.6326489e-04,  4.1578576e-04,  0.0000000e+00], dtype=float32), array([ 0.00436204, 

In [8]:
X_features = np.array([extract_features(x, sr) for x, sr in zip(X, sr_list)])

Extracting Features: 100%|██████████| 4/4 [00:00<00:00, 41.66feature/s]


[[-1.94260147e+02  4.21271515e+01 -1.13794266e+02 ...  9.87792969e-01
   3.10846232e-03 -6.86064124e-01]
 [-1.69822220e+02  4.27622604e+01 -1.14851486e+02 ...  9.87792969e-01
   3.10846232e-03 -6.86064124e-01]
 [-1.88167389e+02  3.93907089e+01 -1.05823494e+02 ...  9.87792969e-01
   3.10846232e-03 -6.86064124e-01]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


Extracting Features: 100%|██████████| 4/4 [00:00<00:00, 52.63feature/s]


[[-265.7307     146.88943    -56.838356  ...   -1.0523673   -0.328352
     0.6098605]
 [-238.03116    151.42467    -71.56885   ...   -1.0523673   -0.328352
     0.6098605]
 [-246.4112     137.24419    -78.372604  ...   -1.0523673   -0.328352
     0.6098605]
 ...
 [   0.           0.           0.        ...    0.           0.
     0.       ]
 [   0.           0.           0.        ...    0.           0.
     0.       ]
 [   0.           0.           0.        ...    0.           0.
     0.       ]]


Extracting Features: 100%|██████████| 4/4 [00:00<00:00, 50.00feature/s]


...

[[-8.0613177e+02 -8.7156315e+00 -1.1566568e+01 ... -1.8027899e-01
   4.6246651e-01  8.3356341e-03]
 [-8.0032947e+02 -1.1651993e+01 -1.6522308e+01 ... -1.8027899e-01
   4.6246651e-01  8.3356341e-03]
 [-8.0152380e+02 -1.0977721e+01 -1.5991901e+01 ... -1.8027899e-01
   4.6246651e-01  8.3356341e-03]
 ...
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]


Extracting Features: 100%|██████████| 4/4 [00:00<00:00, 111.10feature/s]


[[-6.2369415e+02  0.0000000e+00  0.0000000e+00 ...  3.1278621e-02
  -5.0841991e-02 -5.3912770e-02]
 [-6.2369415e+02  0.0000000e+00  0.0000000e+00 ...  3.1278621e-02
  -5.0841991e-02 -5.3912770e-02]
 [-6.2361255e+02 -5.5645086e-02 -6.1739914e-02 ...  3.1278621e-02
  -5.0841991e-02 -5.3912770e-02]
 ...
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]
 [ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ...  0.0000000e+00
   0.0000000e+00  0.0000000e+00]]


Extracting Features: 100%|██████████| 4/4 [00:00<00:00, 102.47feature/s]


[[-9.41950317e+02  4.63755875e+01 -7.72242737e+01 ... -5.23452945e-02
   5.35586417e-01 -1.26169014e+00]
 [-9.43882935e+02  4.62193222e+01 -9.54620819e+01 ... -5.23452945e-02
   5.35586417e-01 -1.26169014e+00]
 [-9.60644592e+02  3.18074398e+01 -1.07536964e+02 ... -5.23452945e-02
   5.35586417e-01 -1.26169014e+00]
 ...
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2, random_state=42)

In [10]:
model = Sequential([
    SimpleRNN(128, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    SimpleRNN(128, activation='relu'),
    Dense(y_train.shape[1], activation='softmax')  # Предполагается, что y_train является one-hot закодированным
])

In [11]:
# Компиляция модели
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
for epoch in tqdm(range(50), desc="Training", unit="epoch"):
    model.fit(X_train, y_train, epochs=1, batch_size=10, validation_data=(X_test, y_test), verbose=0)

Training: 100%|██████████| 50/50 [24:03<00:00, 28.87s/epoch]


In [13]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Accuracy: {accuracy * 100:.2f}%")

9/9 [==============================] - 1s 95ms/step - loss: 2.2751 - accuracy: 0.4021
Accuracy: 40.21%
